In [1]:
import openrouteservice
from bs4 import BeautifulSoup
import requests
import unicodedata
from csv import writer
import re
import pandas as pd
import numpy as np
import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 
import time

#folium
import geopandas as gpd
import re
import folium

In [2]:

output_dir = '../data/raw/'





In [3]:
# use your own key
client = openrouteservice.Client(key='5b3ce3597851110001cf6248f596dd33da864705b4185662da1dd01d')



In [12]:
# read files
domain = pd.read_csv('../data/raw/domain_preprocessed.csv')

# external datasets
hospitals = pd.read_csv('../data/raw/hospital.csv')
care_facility = pd.read_csv('../data/raw/care_facility.csv')

# file with postcode centroid coordinates
centroid = pd.read_csv('../data/raw/postcode_centroid.csv')


In [13]:
#postcode with properties
postcode_list = list(domain['postcode'].unique())

# remove external data not in domain postcode list
hospitals= hospitals.loc[hospitals['postcode'].isin(postcode_list)].reset_index()
care_facility= care_facility.loc[care_facility['postcode'].isin(postcode_list)].reset_index()

In [6]:
"""


hospital


"""

dic_hospital = {'distance': [], 'duration': []}
count = 0

for i in range(len(hospitals)):
    if count/10 == 0:
            time.sleep(5)
    current_postcode = hospitals['postcode'][i]
    txt = hospitals['geometry'][i]
    current_point_txt = re.search("\((.*?)\)",txt)[0]
    current_point = eval(current_point_txt.replace(" ", ","))
    postcode_centroid = eval(centroid[centroid.POSTCODE == current_postcode]['centroid'].values[0])
    route = client.directions(
                coordinates= [postcode_centroid, current_point],
                profile='driving-car',
                format='geojson',
                radiuses = 1000,
            )
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    dic_hospital['distance'].append(distance)
    dic_hospital['duration'].append(duration)
    count += 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

In [8]:
# add distance and duration column to hospital df
hospitals['distance'] = dic_hospital['distance']
hospitals['durations'] = dic_hospital['duration']

In [33]:
# choose hopistal with shortest duration by postcode
code_selected = list(hospitals.groupby('postcode').min('duration')['code'])
hospitals_filtered= hospitals[hospitals.code.isin(code_selected)]

In [34]:
# output directory
output_dir = '../data/raw/'

# output
filename = 'nearest_hospital_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
hospitals_filtered.to_csv(output_dir_full)



In [15]:
'''


care facility


'''

care_facility = care_facility[care_facility.FEATSUBTYP == 'child care'].reset_index()
dic_care_facility = {'distance': [], 'duration': []}
count = 0

for i in range(len(care_facility)):
    if count/10 == 0:
            time.sleep(10)
    current_postcode = care_facility['postcode'][i]
    txt = care_facility['geometry'][i]
    current_point_txt = re.search("\((.*?)\)",txt)[0]
    current_point = eval(current_point_txt.replace(" ", ","))
    postcode_centroid = eval(centroid[centroid.POSTCODE == current_postcode]['centroid'].values[0])
    route = client.directions(
                coordinates= [postcode_centroid, current_point],
                profile='driving-car',
                format='geojson',
                radiuses = 1000,
            )
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    dic_care_facility['distance'].append(distance)
    dic_care_facility['duration'].append(duration)
    count += 1

ApiError: 403 ({'error': 'Quota exceeded'})

In [ ]:
# add distance and duration column to care facility df
care_facility['distance'] = dic_care_facility['distance']
care_facility['durations'] = dic_care_facility['duration']

# choose care_facility with shortest duration by postcode
code_selected = list(care_facility.groupby('postcode').min('duration')['code'])
care_facility_filtered = care_facility[care_facility.code.isin(code_selected)]